<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/TEST_AAE_ROC_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install imageio
!pip install qpsolvers
!pip install shapely 
#!pip install tensorflow_datasets
!pip install keras-layer-normalization
from google.colab import drive
#!pip install alive-progress
drive.mount('/content/drive', force_remount=True)


In [0]:


from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import skimage
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow
	
import statistics
import shutil

import matplotlib.pyplot as plt

import time
#from progress.bar import IncrementalBar

import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, Dropout, Activation, InputLayer
from keras.optimizers import Adam
from keras_layer_normalization import LayerNormalization
from keras.models import load_model
import csv
from scipy.interpolate import make_interp_spline, BSpline

from scipy.signal import savgol_filter

In [0]:
def get_single_test(single_test_path):
    
    sz = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
          sz = sz +1
    test = np.zeros(shape=(sz, conf.dim2, conf.dim3, conf.dim4))
    cnt = 0
    for f in sorted(listdir(single_test_path)):
        if str(join(single_test_path, f))[-3:] == "tif":
            #print("img path: "+join(single_test_path, f))
            img = Image.open(join(single_test_path, f)).resize((conf.dim2, conf.dim3))
            #cv2_imshow(np.array(img,dtype=np.float32))
            #cv2.waitKey(0)
            img = np.array(img, dtype=np.float32) / 256
            test[cnt, :, :, 0] = img
            cnt = cnt + 1
    return test

def get_test_sequences(test_case_dir):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir))
    print("Test case loaded")
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3, conf.dim4))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3, conf.dim4))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :, :]
        sequences[i] = clip
    return sequences





In [0]:
class Config:
    def __init__(self, data_dir_, cwdir_name_, data_set):
        self.data_set_name = data_set
        self.data_dir = data_dir_
        self.data_set_dir = join(self.data_dir, data_set)
        self.cwdir_name = cwdir_name_
        self.cwdir = join(self.data_dir,self.cwdir_name)
        self.run_data = join(self.cwdir, "training_dir")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        if not os.path.exists(self.cwdir):
            os.mkdir(self.cwdir)
            os.mkdir(self.run_data)
    
        if not os.path.exists(self.run_data):
            #shutil.rmtree(self.run_data)
            os.mkdir(self.run_data)
            os.makedirs(self.image_dir, exist_ok=True)

        self.DATASET_PATH = join(self.data_set_dir,"Train/")
        self.TEST_DIR = join(self.data_set_dir,"Test/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 50
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_AAE")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_AAE")
        self.DEC_MODEL_PATH = join(self.cwdir,"model_dec_Conv2DLSTM_AAE")

        self.retrain = 0
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.latent_dim = 327680


    def reconfig(self, new_name, batch_size = 4, epochs = 5, retrain = 0):
        self.cwdir_name = new_name
        self.cwdir = join(self.data_dir, self.cwdir_name)
        self.run_data = join(self.cwdir, "training_dir")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")

        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_AAE")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_AAE")
        self.DEC_MODEL_PATH = join(self.cwdir,"model_dec_Conv2DLSTM_AAE")

        self.retrain = retrain
        if retrain == 0:
            print("Configuring train from scratch")
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
    
            if os.path.exists(self.run_data):
                shutil.rmtree(self.run_data)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)


conf = Config(data_dir_="/content/drive/My Drive/", cwdir_name_="Conv2DLSTM_AAE_PED1", data_set="UCSD_Anomaly_Dataset.v1p2/UCSDped1/") 


In [0]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/content/drive/My Drive/Persistence1D/python')


import matplotlib.pyplot as plt
import numpy as np
from persistence1d import RunPersistence
from reconstruct1d import RunReconstruction
import math    

from shapely.geometry import LineString

In [0]:
TestVideoFile_ped1 = list((36,2))
end = 190
TestVideoFile_ped1 = [[60,152], [50,175], [91,end], [31,168], [5,90], [1,100], [1,175], [1,94], [1,48], [1,140],   [70,165],   [130,end],   [1,156],   [1,end],   [138,end],   [123,end],   [1,47],   [54,120],    [64,138],    [45,175],    [31,end],    [16,107],    [8,165],    [50,171],    [40,135],    [77,144],    [10,122],    [105,end],    [1,15],    [175,end],    [1,180],    [1,52],  [5,165],    [1,121],    [86,end],   [15,108]]




def SortExtremaByPersistence(ExtremaAndPersistence):
    #~ Sort the list of extrema by persistence.
    #~ The original list from RunPersistence() is not guaranteed to be sorted,
    #~ although it may appear sorted in many cases.
    #~ This call to sorted() creates a new list. If you want to sort in-place, use ExtremaAndPersistence.sort()
    SortedExtremaAndPersistence = sorted(ExtremaAndPersistence, key=lambda ExtremumAndPersistence: ExtremumAndPersistence[0])
    return SortedExtremaAndPersistence

def GetMinima(ExtremaAndPersistence):
    Minima = [t for t in ExtremaAndPersistence[::2]]
    return Minima

def GetMaxima(ExtremaAndPersistence):
    Maxima = [t for t in ExtremaAndPersistence[1::2]]
    return Maxima


def GetIntersection(x, f, g):

    plt.plot(x, f)
    plt.plot(x, g)

    first_line = LineString(np.column_stack((x, f)))
    second_line = LineString(np.column_stack((x, g)))
    intersection = first_line.intersection(second_line)
    print("Intersection")
    #print(intersection)
    x_list = []
    if intersection.geom_type == 'MultiPoint':
        plt.plot(*LineString(intersection).xy, 'o')
        print("Multipoint")
        print(*LineString(intersection).xy[0])
        for x_point in LineString(intersection).xy[0]:
            x_list.append(x_point)
    elif intersection.geom_type == 'Point':
        plt.plot(*intersection.xy, 'o')
        print("Point")
        print(*intersection.xy[0])
        x_list.append(*intersection.xy[0])

    x_arr = np.array(x_list)
    #print("X_LIST")
    #print(x_arr)
    return x_arr
    #all(i >= 30 for i in g[])

def find_overlap_rate(reference, detection):

    overlap = 0
    total_ref = 0
    for x, y in detection:

        print("Intersect x, y   =   "+str(x)+" , "+str(y))
        for p, q in reference:
            print("Reference p, q   =   "+str(p)+" , "+str(q))
            if y > p and x < q:
                overlap = overlap+ abs(min(y,q) - max(x,p))

    for p, q in reference:
        total_ref = total_ref+ abs(q-p)
    
    print("Total Ref: "+str(total_ref))
    print("Overlap: "+str(overlap))
    retVal = overlap
    if retVal > 0 :
        retVal = retVal/total_ref
    return retVal

def get_TPR_FPR(intersections, threshold_abs, gt_tuples, all_below_th):
    print("_________Find TPR FPR for Threshold : "+str(threshold_abs)+"__________")
    print("============Find TPR============")
    TPR = find_overlap_rate(gt_tuples, all_below_th)
    
    gt_inverse_tuples = []

    tf_start = 0
    tf_abs_end = intersections[intersections.shape[0]-1]
    for p, q in gt_tuples:
        if(p == tf_start):
            tf_start = q
            continue
        gt_inverse_tuples.append((tf_start, p))
        tf_start = q

    if tf_start < tf_abs_end:
        gt_inverse_tuples.append((tf_start, tf_abs_end))
    print(gt_inverse_tuples)
    print("============Find FPR============")
    FPR = find_overlap_rate(gt_inverse_tuples, all_below_th)

    return TPR, FPR

def persistence(InputData, dt, id, color):
    #~ Compute the extrema of the given data and their persistence.
    ExtremaAndPersistence = RunPersistence(InputData)

    #~ Keep only those extrema with a persistence larger than 0.5.
    FilteredIndices = [t[0] for t in ExtremaAndPersistence if t[1] >= 0.01]

    #~ This simple call is all you need to reconstruct a smooth function containing only the filtered extrema
    SmoothData = RunReconstruction(InputData, FilteredIndices, 'biharmonic', 0.0000001)
    #print("Smooth data ")
    #print(SmoothData)

    x = np.array([x for x in range(0, InputData.shape[0])])
    
    g = SmoothData


    ##add gt_tuples : TODO
    
    #print(intersections)
    #~ Plot original and smoothed data
    fig, ax = plt.subplots()
    ax.plot(range(0, len(InputData)), InputData, label="Original Data")
    ax.plot(range(0, len(SmoothData)), SmoothData, label="Smooth Data")
    ExtremaIndices = [t[0] for t in ExtremaAndPersistence]
    
    """
    sorted_extr = SortExtremaByPersistence(ExtremaAndPersistence)
    minima = SortExtremaByPersistence(GetMinima(ExtremaAndPersistence))
    maxima = SortExtremaByPersistence(GetMaxima(ExtremaAndPersistence))
    print("###########Print All Extremas#################")

  
    print([t for t in ExtremaAndPersistence])

    print("###########Print Filtered Extremas#################")

    print([t for t in FilteredIndices])

    print("###########Print Sorted Extremas#################")

    print([t for t in sorted_extr])

    print("###########Print Minimas#################")

    print([t for t in minima])

    print("###########Print Maximas#################")

    print([t for t in maxima])
    """


    gt_tuples = []
    ax.plot(ExtremaIndices, InputData[ExtremaIndices], marker='.', linestyle='')
    ax.plot(FilteredIndices, InputData[FilteredIndices], marker='*', linestyle='')
    ax.set(xlabel='data index', ylabel='data value')
    #ax.set_aspect(1.0/ax.get_data_ratio()*0.2)
    plt.axvspan(TestVideoFile_ped1[id-1][0]-1, TestVideoFile_ped1[id-1][1]-1, alpha=0.5, color=color)
    #plt.axhline(y=threshold_abs,linewidth=1, color='blue') 


    gt_tuples.append((TestVideoFile_ped1[id-1][0]-1, TestVideoFile_ped1[id-1][1]-1))
    if dt == 1 and id == 5:
        plt.axvspan(140-1, end-1, alpha=0.5, color=color)
        gt_tuples.append((140-1, end-1))
    elif dt == 1 and id == 6:
        plt.axvspan(110-1, end-1, alpha=0.5, color=color) 
        gt_tuples.append((110-1, end-1))
    elif dt == 1 and id == 29:
        plt.axvspan(45-1, 113-1, alpha=0.5, color=color)
        gt_tuples.append((45-1, 113-1))
    elif dt == 1 and id == 32:
        plt.axvspan(65-1, 115-1, alpha=0.5, color=color)
        gt_tuples.append((65-1, 115-1))


    TPR_ARR = []
    FPR_ARR = []

    for threshold_abs in np.arange(0.6, 1.05, 0.05):
        f = np.ones(x.shape[0])*threshold_abs
        intersections = [0]
        intersections.extend(GetIntersection(x, f, g))
        if(intersections[len(intersections)-1] < end -1):
            intersections.extend([end-1])

        intersections = np.array(intersections)
        all_below_th = []
        #all_above_th = []

        for j, __ in enumerate(intersections):

            if all(i <= threshold_abs for i in g[math.ceil(intersections[j]): math.floor(intersections[j+1])]):
                all_below_th.append((intersections[j], intersections[j+1]))
                
            if(end - 1 == math.floor(intersections[j+1])):
                break
            #if all(i >= threshold_abs for i in g[math.ceil(intersections[j]): math.floor(intersections[j+1])]):
            #    all_above_th.append((intersections[j], intersections[j+1]))


        TPR, FPR = get_TPR_FPR(intersections, threshold_abs, gt_tuples, all_below_th)
        
        TPR_ARR.append(TPR)
        FPR_ARR.append(FPR)
    
        print("TPR : "+str(TPR))
        print("FPR : "+str(FPR))
        print("\n\n\n")
    
    ax.grid()
    #ax.plot()

    plt.legend()    
    plt.show()

    return TPR_ARR, FPR_ARR

threshold = 0.9
def fill_gt_ped1(sr, id, color):
    #plt.xlim((conf.dim1,len(sr)+conf.dim1))
    sr = np.reshape(sr, (sr.shape[0],))
    print(sr.shape)
    #zeros = np.zeros((conf.dim1,))
    #print(zeros.shape)
    #plt.plot(np.concatenate((zeros, sr)))
    #persistence(sr, 1, id, color)
    yhat = savgol_filter(sr, 51, 3)
    TPR_ARR, FPR_ARR = persistence(yhat, 1, id, color)
    #plt.plot(sr)  
    return TPR_ARR, FPR_ARR

    #plt.plot(yhat, color='red')
    plt.axvspan(TestVideoFile_ped1[id-1][0], TestVideoFile_ped1[id-1][1], alpha=0.5, color=color)
    #plt.axhline(y=threshold_abs,linewidth=1, color='blue')
    if id == 5:
        plt.axvspan(140, 200, alpha=0.5, color=color)
    elif id == 6:
        plt.axvspan(110, 200, alpha=0.5, color=color) 
    elif id == 29:
        plt.axvspan(45, 113, alpha=0.5, color=color)
    elif id == 32:
        plt.axvspan(65, 115, alpha=0.5, color=color)


    plt.ylabel('regularity score sr_ae(t)')
    plt.xlabel('frame t')
    plt.show()



TestVideoFile_ped2 = list((12,2))
TestVideoFile_ped2 = [[61,180],[95,180],[1,146],[31,180],[1,129],[1,159],[46,180],[1,180],[1,120],[1,150],[1,180],[88,180]]

def fill_gt_ped2(sr, id, color):
    #plt.xlim((conf.dim1,len(sr)+conf.dim1))
    sr = np.reshape(sr, (sr.shape[0],))
    #plt.plot(np.concatenate((np.zeros((conf.dim1,)), sr)))
    persistence(sr, 2, id, color)
    yhat = savgol_filter(sr, 51, 3)
    TPR_ARR, FPR_ARR = persistence(yhat, 2, id, color)
    return TPR_ARR, FPR_ARR

    #plt.plot(sr)

    #plt.plot(yhat, color='red')
    plt.axvspan(TestVideoFile_ped2[id-1][0], TestVideoFile_ped2[id-1][1], alpha=0.5, color=color)
    #plt.axhline(y=threshold_abs,linewidth=1, color='blue')
    plt.ylabel('regularity score sr_ae(t)')
    plt.xlabel('frame t')
    plt.show()



In [0]:
def build_model_aae():
  
    model_enc = load_model(conf.cwdir+"/model_gen_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})
    model_dec = load_model(conf.cwdir+"/model_dec_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})
    model_disc = load_model(conf.cwdir+"/model_dis_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})

    #model_enc = load_model("/content/drive/My Drive/model_gen_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})
    #model_dec = load_model("/content/drive/My Drive/model_dec_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})
    #model_disc = load_model("/content/drive/My Drive/model_dis_Conv2DLSTM_AAEep100",custom_objects={'LayerNormalization': LayerNormalization})

    model_ae = Sequential()
    model_ae.add(model_enc)
    model_ae.add(model_dec)
    
    model_enc_disc = Sequential()
    model_enc_disc.add(model_enc)
    model_enc_disc.add(model_disc)
    
    return model_enc, model_dec, model_disc, model_ae, model_enc_disc

model_enc, model_dec, model_disc, model_ae, model_enc_disc = build_model_aae()

model_enc.summary()
model_dec.summary()
model_disc.summary()
model_ae.summary()
model_enc_disc.summary()

model_disc.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy")
model_enc_disc.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy")
model_ae.compile(optimizer=Adam(lr=1e-4, decay=1e-5, epsilon=1e-6), loss="mse")
#"/content/drive/My Drive/UCSD_Anomaly_Dataset.v1p2/model.hdf5"

#model_ae = load_model("/content/drive/My Drive/VAE/model1.hdf5",custom_objects={'LayerNormalization': LayerNormalization})

def evaluate_dis(sequences, model, id, dt):
    fooling_loss = model.predict(sequences,batch_size=conf.BATCH_SIZE)
    sa = (fooling_loss - np.min(fooling_loss)) / (np.max(fooling_loss))
    sr = 1.0 - sa

    with open(join("/content/drive/My Drive/", 'sr_score.csv'), mode='a') as f:
        writer = csv.writer(f)
        writer.writerow([float(x[0]) for x in sr])
        f.close()

    threshold_abs = threshold# np.min(sr)+ (np.max(sr) - np.min(sr)) * threshold
    if dt == 1:
        fill_gt_ped1(sr, id, 'red', threshold_abs)
    elif dt == 2:
        fill_gt_ped2(sr, id, 'red', threshold_abs)
    #plt.plot(sr)
    #plt.ylabel('regularity score sr_dis(t)')
    #plt.xlabel('frame t')
    #plt.show()

def get_clips(test):
    sz = test.shape[0] - conf.dim1
    sequences = np.zeros((sz, conf.dim1, conf.dim2, conf.dim3))
    # apply the sliding window technique to get the sequences
    for i in range(0, sz):
        clip = np.zeros((conf.dim1, conf.dim2, conf.dim3))
        for j in range(0, conf.dim1):
            clip[j] = test[i + j, :, :]
        sequences[i] = clip
    return sequences


def mask_array(bg_list):

    for p in range(0, bg_list.shape[0]):
        for r in range(256):
            for c in range(256):
                if bg_list[p][r][c] > 0:
                    bg_list[p][r][c] = 1
                else:
                    bg_list[p][r][c] = 0

    bg_list[0, :, :] = 0
    return bg_list

def evaluate_ae(sequences, model, id, dt, bg_list=[], mask=1):
  
    sz = len(sequences)

    reconstructed_sequences = model.predict(sequences,batch_size=conf.BATCH_SIZE)
    #print(bg_list[10])

    masked_bg = mask_array(bg_list)
    bg_clips = get_clips(masked_bg)

    sequences = np.reshape(sequences, (sequences.shape[0], sequences.shape[1], sequences.shape[2], sequences.shape[3]))
    reconstructed_sequences = np.reshape(reconstructed_sequences, (reconstructed_sequences.shape[0], reconstructed_sequences.shape[1], reconstructed_sequences.shape[2], reconstructed_sequences.shape[3]))

    if mask == 1:
        print("==============With BG Masked==============")
        sq_masked = np.multiply(sequences, bg_clips)
        rec_masked = np.multiply(reconstructed_sequences, bg_clips)
    else:
        print("==============Without BG Masked==============")
        sq_masked = sequences
        rec_masked = reconstructed_sequences

    #bg_weights = np.add(np.ones(bg_clips.shape), bg_clips)
    sequences_reconstruction_cost = np.array([np.linalg.norm(np.subtract(sq_masked[i], rec_masked[i])) for i in range(0,sz)])
    #sequences_reconstruction_cost = np.array([np.linalg.norm(np.multiply(bg_weights[i],np.subtract(sequences[i], reconstructed_sequences[i]))) for i in range(0,sz)])


    sa = (sequences_reconstruction_cost - np.min(sequences_reconstruction_cost)) / (np.max(sequences_reconstruction_cost))

    
    sr = 1 - sa
    #threshold_abs = threshold#np.min(sr)+ (np.max(sr) - np.min(sr)) * threshold
    if dt == 1:
        TPR_ARR, FPR_ARR = fill_gt_ped1(sr, id, 'yellow')
    elif dt == 2:
        TPR_ARR, FPR_ARR = fill_gt_ped2(sr, id, 'yellow')
    #plt.plot(sr)
    #plt.ylabel('regularity score sr_ae(t)')
    #plt.xlabel('frame t')
    #plt.show()
    return TPR_ARR, FPR_ARR

def load_input_data_list():
    path = join("/content/drive/My Drive/", 'sr_score.csv')
    InputDataList = LoadData(path)
    return InputDataList


def get_persistance(InputData):
  #~ This simple call is all you need to compute the extrema of the given data and their persistence.
  ExtremaAndPersistence = RunPersistence(InputData)

  #~ Keep only those extrema with a persistence larger than 10.
  Filtered = [t for t in ExtremaAndPersistence if ExtremaAndPersistence[1] > 50]
  print(ExtremaAndPersistence)
  print(Filtered)
  #~ Sort the list of extrema by persistence.
  #Sorted = sorted(Filtered, key=lambda ExtremumAndPersistence: ExtremumAndPersistence[1])

  return Filtered


conf.reconfig(new_name="Conv2DLSTM_AAE_PED1", batch_size=4, epochs=100, retrain=1)




def edge_detect(gray):
    edges1 = np.uint8(gray)
    # Using the Canny filter to get contours
    #edges = cv2.Canny(edges1, 20, 30)
    
    # Using the Canny filter with different parameters
    edges_high_thresh = cv2.Canny(edges1, 160, 200)
    kernel = np.zeros((5,5),np.uint8)
    kernel2 = np.zeros((2,2),np.uint8)

    images = cv2.morphologyEx(edges_high_thresh, cv2.MORPH_OPEN, kernel)
    images = cv2.morphologyEx(edges_high_thresh, cv2.MORPH_OPEN, kernel)
    images = cv2.morphologyEx(images, cv2.MORPH_CLOSE, kernel2)
    # Stacking the images to print them together
    # For comparison
    #images = np.hstack((gray, edges, edges_high_thresh))

    #kernel = np.ones((3,3), np.uint8) 
    #fg_mask = cv2.erode(edges_high_thresh, kernel, iterations=2)
    # Display the resulting frame
    #cv2_imshow(images)
    return images

    

def connected_comp(img):

    #find all your connected components (white blobs in your image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    #connectedComponentswithStats yields every seperated component with information on each of them, such as size
    #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
    sizes = stats[1:, -1]; nb_components = nb_components - 1

    # minimum size of particles we want to keep (number of pixels)
    #here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever
    min_size = 10  

    #your answer image
    img2 = np.zeros((output.shape))
    #for every component in the image, you keep it only if it's above min_size
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img2[output == i + 1] = 255
    return img2

def get_background_subtr(test_case):
    print("BGS show")
    #subtractor = cv2.createBackgroundSubtractorMOG2(history=30, varThreshold=50, detectShadows=True)
    bg_list = []
    for frame in test_case:
        frame = np.reshape(frame, (256, 256))*256
        edge_mask = edge_detect(frame)
        bg_mask = connected_comp(edge_mask)
        #bg_mask = subtractor.apply(edge_mask)
        #edge_mask[0] = np.zeros(edge_mask[0].shape)
        #kernel = np.ones((5,5), np.uint8) 
        #fg_mask = cv2.erode(mask, kernel, iterations=2)
        #fg_mask = cv2.dilate(fg_mask, kernel, iterations=5)
        fg_mask = bg_mask/255.0
        #fg_mask = mask
        #cv2_imshow(fg_mask)
        #print(fg_mask)
        bg_list.append(fg_mask)
        #key = cv2.waitKey(30)
        #if key == 27:
        #    break
    cv2_imshow(bg_list[100]*255.0)
    #cv2.destroyAllWindows()
    #print(bg_list)
    return np.array(bg_list)


def show_bgs(test_case_dir):
    test = get_single_test(join(conf.TEST_DIR,test_case_dir))
    return get_background_subtr(test)


TPR_FOR_CASES = []
FPR_FOR_CASES = []
#print(bg_list[10])
for i in range(1,37):
  if i < 10:
    img_num = "00"+str(i)
  elif i < 100:
    img_num = "0"+str(i)
  else:
    img_num = str(i) 

  #if img_num == "017":
  #  continue

  bg_list = show_bgs("Test"+img_num)
  
  
  test_cases_dir = "Test"+img_num
  test_cases = get_test_sequences(test_cases_dir)
  print("Test"+img_num+" data set loaded")
  #evaluate_dis(test_cases, model_enc_disc, i, 1)
  #sorted_sr = get_persistance(np.array([x[0] for x in sr_dis], dtype=float))
  #print(sorted_sr)
  #break
  TPR_ARR, FPR_ARR = evaluate_ae(test_cases, model_ae, i, 1, bg_list, 1)
  TPR_FOR_CASES.append(TPR_ARR)
  FPR_FOR_CASES.append(FPR_ARR)
  #evaluate_ae(test_cases, model_ae, i, 1, bg_list, 0)

  #sr_comb = (sr_dis + sr_ae)*0.5
  #sr_comb = []

  continue
TPR_FOR_CASES = numpy.array(TPR_FOR_CASES)
FPR_FOR_CASES = numpy.array(FPR_FOR_CASES)

TPR_PLOT = numpy.mean(TPR_FOR_CASES, axis=0)
FPR_PLOT = numpy.mean(FPR_FOR_CASES, axis=0)

print(TPR_PLOT.shape)
print(FPR_PLOT.shape)

plt.plot(TPR_PLOT)
plt.plot(FPR_PLOT)
plt.show()